**1321. Restaurant Growth**<br><br>
You are the restaurant owner and you want to analyze a possible expansion (there will be at least one customer every day).

Write an SQL query to compute moving average of how much customer paid in a 7 days window (current day + 6 days before) .

The query result format is in the following example:

Return result table ordered by visited_on.

average_amount should be rounded to 2 decimal places

In [ ]:
# Method 1: Windows Function Application with Case used 2 times

SELECT t.visited_on, t.amount, ROUND(t.Average,2) AS average_amount 
FROM(
SELECT visited_on,
    CASE WHEN ROW_NUMBER() OVER (ORDER BY visited_on) >= 7 THEN SUM(SUM(amount)) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) END AS amount, 
    CASE WHEN ROW_NUMBER() OVER (ORDER BY visited_on) >= 7 THEN AVG(SUM(amount)) OVER(ORDER BY visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) END AS Average
FROM Customer
GROUP BY visited_on
ORDER BY visited_on) t
WHERE t.amount IS NOT NULL

In [ ]:
# Method 2: Windows Function Application with Subqueries

SELECT T2.visited_on, T2.amount, T2.average_amount
FROM (SELECT visited_on,
        SUM(amount) OVER (ORDER BY visited_on ROWS 6 PRECEDING) AS amount,
        ROUND(AVG(amount) OVER (ORDER BY visited_on ROWS 6 PRECEDING),2) AS average_amount, 
        ROW_NUMBER() OVER (ORDER BY visited_on) AS r_num 
    FROM (SELECT visited_on, SUM(amount) AS amount FROM customer GROUP BY visited_on ORDER BY visited_on) AS T1) AS T2
WHERE T2.r_num >= 7

**1341. Movie Rating**<br>
Write the following SQL query:

Find the name of the user who has rated the greatest number of movies.
In case of a tie, return lexicographically smaller user name.

Find the movie name with the highest average rating in February 2020.
In case of a tie, return lexicographically smaller movie name.

In [ ]:
(SELECT u.name AS results
 FROM Users u
 JOIN Movie_Rating mr
 ON u.user_id = mr.user_id
 GROUP BY u.name
 ORDER BY COUNT(mr.user_id) DESC, u.name
 LIMIT 1)

UNION

(SELECT m.title AS results
 FROM Movies m
 JOIN Movie_Rating mr
 ON m.movie_id = mr.movie_id
 WHERE mr.created_at LIKE '2020-02%'
 GROUP BY m.title
 ORDER BY AVG(mr.rating) DESC, m.title
 LIMIT 1)

**1355. Activity Participants**<br>
Write an SQL query to find the names of all the activities with neither maximum, nor minimum number of participants.

In [ ]:
WITH act AS
    (SELECT f.activity, COUNT(*) AS cnt
     FROM Friends f
     GROUP BY f.activity)

SELECT activity AS activity
FROM act
WHERE cnt > (SELECT MIN(cnt) FROM act) AND 
      cnt < (SELECT MAX(cnt) FROM act)

**1364. Number of Trusted Contacts of a Customer**<br>
Write an SQL query to find the following for each invoice_id:
1. customer_name
2. price
3. contacts_cnt
4. trusted_contacts_cnt<br>

Order the result table by invoice_id.

In [ ]:
SELECT t1.invoice_id, 
       t1.customer_name, 
       t1.price, 
        IFNULL(t2.contacts_cnt,0) AS contacts_cnt, 
        IFNULL(t2.trusted_contacts_cnt,0) AS trusted_contacts_cnt
FROM
    (SELECT i.invoice_id, i.user_id, cu.customer_name, i.price
    FROM Invoices i
    JOIN Customers cu
    ON i.user_id = cu.customer_id) t1
LEFT JOIN
    (SELECT co.user_id, COUNT(*) AS contacts_cnt, COUNT(cus.customer_id) AS trusted_contacts_cnt
    FROM Customers cus
    RIGHT JOIN Contacts co
    ON cus.email = co.contact_email
    GROUP BY co.user_id) t2
ON t1.user_id = t2.user_id
ORDER BY t1.invoice_id

**1393. Capital Gain/Loss**<br>
Write an SQL query to report the Capital gain/loss for each stock.

The capital gain/loss of a stock is total gain or loss after buying and selling the stock one or many times.

In [ ]:
SELECT stock_name,
       SUM(CASE WHEN operation = "Sell" THEN price ELSE 0 END) - SUM(CASE WHEN operation = "Buy" THEN price ELSE 0 END) 
       AS capital_gain_loss
FROM Stocks
GROUP BY stock_name;

**1398. Customers Who Bought Products A and B but Not C**<br>
Write an SQL query to report the customer_id and customer_name of customers who bought products "A", "B" but did not buy the product "C" since we want to recommend them buy this product.

Return the result table ordered by customer_id.

In [ ]:
# Method 1: Via Case Function

SELECT t.customer_id, t.customer_name
FROM
    (SELECT c.customer_id, c.customer_name,
            SUM(CASE WHEN o.product_name = 'A' THEN 1
                     WHEN o.product_name = 'B' THEN 1
                     WHEN o.product_name = 'C' THEN -1 ELSE 0 END) AS calc
    FROM Customers c
    JOIN Orders o
    ON c.customer_id = o.customer_id
    GROUP BY c.customer_id) t
WHERE t.calc > 1

In [ ]:
# Method 2: Via Group Concat method 

SELECT t.customer_id, t.customer_name
FROM
    (SELECT c.customer_id, c.customer_name, 
            GROUP_CONCAT(DISTINCT o.product_name ORDER BY o.product_name) AS products
FROM Customers c
JOIN Orders o
ON c.customer_id = o.customer_id
GROUP BY c.customer_id) t
WHERE t.products LIKE 'A,B%' AND t.products NOT LIKE '%C%'

**1421. NPV Queries**<br>
Write an SQL query to find the npv of all each query of queries table.

In [ ]:
SELECT q.id, q.year, IFNULL(n.npv, 0) AS npv
FROM Queries q
LEFT JOIN npv n
ON q.id = n.id AND q.year = n.year

**1440. Evaluate Boolean Expression**<br>
Write a SQL query to evaluate the boolean expressions in Expressions table.

In [ ]:
SELECT e.left_operand, e.operator, e.right_operand,
CASE WHEN e.operator = '>' AND v1.value>v2.value THEN 'true'
WHEN e.operator = '<' AND v1.value<v2.value THEN 'true'
WHEN e.operator = '=' AND v1.value=v2.value THEN 'true' ELSE 'false' END AS value
FROM Expressions e
JOIN Variables v1
ON e.left_operand = v1.name
JOIN Variables v2
ON e.right_operand = v2.name

**1445. Apples & Oranges**<br>
Write an SQL query to report the difference between number of apples and oranges sold each day.

Return the result table ordered by sale_date in format ('YYYY-MM-DD').

In [ ]:
# Method 1: Self Join

SELECT a.sale_date, b.sold_num-a.sold_num AS diff
FROM Sales a, Sales b
WHERE a.sale_date = b.sale_date
GROUP BY 1
ORDER BY 1

In [ ]:
# Method 2: Case-Function

SELECT sale_date, 
       SUM(CASE WHEN fruit = 'apples' THEN sold_num ELSE -sold_num END) AS diff
FROM sales
GROUP BY sale_date
ORDER BY sale_date

**1454. Active Users**<br>
Write an SQL query to find the id and the name of active users.<br>
Active users are those who logged in to their accounts for 5 or more consecutive days.<br>
Return the result table ordered by the id.

In [ ]:
SELECT t.id, ac.name
FROM Accounts ac
JOIN
(SELECT DISTINCT a.id FROM logins a, logins b 
WHERE a.id=b.id AND DATEDIFF(a.login_date, b.login_date) BETWEEN 1 AND 4
GROUP BY a.id, a.login_date
HAVING COUNT(DISTINCT b.login_date) = 4) t
ON t.id = ac.id
ORDER BY t.id

**1459. Rectangles Area**<br>
Write an SQL query to report of all possible rectangles which can be formed by any two points of the table. 

Each row in the result contains three columns (p1, p2, area) where:

p1 and p2 are the id of two opposite corners of a rectangle and p1 < p2.
Area of this rectangle is represented by the column area.
Report the query in descending order by area in case of tie in ascending order by p1 and p2.

In [ ]:
SELECT a.id AS p1, 
       b.id AS p2, 
       ABS(a.x_value - b.x_value)*ABS(a.y_value - b.y_value) AS area
FROM Points a, Points b
WHERE a.id < b.id
HAVING area != 0
ORDER BY 3 DESC, 1,2

**1468. Calculate Salaries**<br>
Write an SQL query to find the salaries of the employees after applying taxes.

The tax rate is calculated for each company based on the following criteria:
1. 0% If the max salary of any employee in the company is less than 1000 USD.
2. 24% If the max salary of any employee in the company is in the range [1000, 10000] inclusive.
3. 49% If the max salary of any employee in the company is greater than 10000 USD.

Return the result table in any order. Round the salary to the nearest integer.

In [ ]:
WITH taxcalc AS (
SELECT company_id, 
CASE WHEN MAX(salary)>10000 THEN 0.51
WHEN MAX(salary)<1000 THEN 1.00 ELSE 0.76 END AS num
FROM salaries
GROUP BY company_id)

SELECT s.company_id, s.employee_id, s.employee_name, ROUND(s.salary*t.num) AS salary
FROM Salaries s
JOIN taxcalc t
ON s.company_id = t.company_id

**1501. Countries You Can Safely Invest In**<br>
A telecommunications company wants to invest in new countries. The company intends to invest in the countries where the average call duration of the calls in this country is strictly greater than the global average call duration.

Write an SQL query to find the countries where this company can invest.

In [ ]:
SELECT c.name AS country FROM Person p
JOIN Country c
ON c.country_code = SUBSTRING(p.phone_number, 1, 3)
JOIN Calls ca
ON ca.caller_id = p.id OR ca.callee_id = p.id
GROUP BY c.name
HAVING AVG(duration) > (SELECT AVG(duration) FROM calls)

**1532. The Most Recent Three Orders**<br>
Write an SQL query to find the most recent 3 orders of each user. If a user ordered less than 3 orders return all of their orders.

Return the result table sorted by customer_name in ascending order and in case of a tie by the customer_id in ascending order. If there still a tie, order them by the order_date in descending order.

In [ ]:
SELECT t.name AS customer_name, t.customer_id, t.order_id, t.order_date
FROM (SELECT c.name, 
             c.customer_id, 
             o.order_id, 
             o.order_date, 
             ROW_NUMBER() OVER (PARTITION BY o.customer_id ORDER BY o.order_date DESC) AS rown
             FROM Customers c
             JOIN Orders o
             ON c.customer_id = o.customer_id) t
WHERE t.rown<=3
ORDER BY customer_name, customer_id, order_date DESC

**1549. The Most Recent Orders for Each Product**<br>
Write a SQL query to find the most recent order(s) of each product.<br>

Return the result table sorted by product_name in ascending order and in case of a tie by the product_id in ascending order.<br> If there still a tie, order them by the order_id in ascending order.

In [ ]:
# Method 1:

SELECT p.product_name, o.product_id, o.order_id, o.order_date
FROM Orders o 
JOIN Products p
ON o.product_id = p.product_id
WHERE (o.order_date, o.product_id) 
IN (SELECT MAX(order_date) AS order_date, product_id FROM Orders GROUP BY product_id)
ORDER BY p.product_name, o.product_id, o.order_id

In [ ]:
# Method 2:

SELECT t.product_name, t.product_id, t.order_id, t.order_date
FROM (SELECT p.product_name, 
             o.product_id, 
             o.order_id, 
             o.order_date, 
             RANK() OVER (PARTITION BY o.product_id ORDER BY o.order_date DESC) AS rnk
      FROM Products p JOIN Orders o
      ON o.product_id = p.product_id) t
WHERE t.rnk = 1
ORDER BY t.product_name, t.product_id, t.order_id

**1555. Bank Account Summary**<br>
Write an SQL query to report user_id, user_name, credit (current balance after performing transactions),<br>
credit_limit_breached (check credit_limit ("Yes" or "No"))

In [ ]:
SELECT u.user_id, u.user_name, 
       IFNULL((SUM(t.amount) + u.credit),u.credit) AS credit,
       (CASE WHEN IFNULL((SUM(t.amount) + u.credit),u.credit)<0 THEN 'Yes' ELSE 'No' END) AS credit_limit_breached
FROM
    (SELECT paid_by AS user_id, SUM((-1)*amount) AS amount FROM transaction GROUP BY user_id
    UNION
    SELECT paid_to AS user_id, SUM(amount) FROM transaction GROUP BY user_id) t
RIGHT JOIN Users u 
ON u.user_id = t.user_id
GROUP BY u.user_id

**1596. The Most Frequently Ordered Products for Each Customer**<br>
Write an SQL query to find the most frequently ordered product(s) for each customer.

The result table should have the product_id and product_name for each customer_id who ordered at least one order.<br> Return the result table in any order.

In [ ]:
SELECT t2.customer_id, t2.product_id, p.product_name 
FROM(
    SELECT t.customer_id, t.product_id, RANK() OVER(PARTITION BY t.customer_id ORDER BY t.ctpid DESC) AS rnk 
    FROM(
        SELECT customer_id, product_id, COUNT(product_id) AS ctpid 
        FROM Orders 
        GROUP BY customer_id, product_id) t) t2
JOIN Products p
ON p.product_id = t2.product_id
WHERE t2.rnk = 1

**1613. Find the Missing IDs**<br>
Write an SQL query to find the missing customer IDs. The missing IDs are ones that are not in the Customers table but are in the range between 1 and the maximum customer_id present in the table.

Notice that the maximum customer_id will not exceed 100.

Return the result table ordered by ids in ascending order.

In [ ]:
WITH RECURSIVE id_seq AS (
    SELECT 1 as continued_id
    UNION 
    SELECT continued_id + 1
    FROM id_seq
    WHERE continued_id < (SELECT MAX(customer_id) FROM Customers) 
)

SELECT continued_id AS ids
FROM id_seq
WHERE continued_id NOT IN (SELECT customer_id FROM Customers)